In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


input_shape = (224, 224, 3)

#Instantiate an empty model
model = Sequential([
Conv2D(64, (3, 3), input_shape=input_shape, padding='same', activation='relu'),
Conv2D(64, (3, 3), activation='relu', padding='same'),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(128, (3, 3), activation='relu', padding='same'),
Conv2D(128, (3, 3), activation='relu', padding='same'),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(256, (3, 3), activation='relu', padding='same',),
Conv2D(256, (3, 3), activation='relu', padding='same',),
Conv2D(256, (3, 3), activation='relu', padding='same',),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Flatten(),
Dense(1024, activation='relu'),
Dense(1024, activation='relu'),
Dense(2, activation='softmax')
])

model.summary()

# Compile the model
#model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=["accuracy"])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 56, 56, 256)      

In [ ]:
batch_size = 32
train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/Project/Data/CNNData/Train',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        '/content/drive/My Drive/Project/Data/CNNData/Test',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

save_callback = ModelCheckpoint('CNN_Models_VGGNet_save_best_epoch.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='min', period=10)

class CustomModelCheckpoint(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # logs is a dictionary
        if epoch % 10 == 0:
          #print(f"epoch: {epoch}, train_acc: {logs['acc']}, valid_acc: {logs['val_acc']}")
          #if logs['val_acc'] > logs['acc']: # your custom condition
          self.model.save('CNN_Models_VGGNet_' + str(epoch) + '_model.h5')

save_per_epoch = CustomModelCheckpoint()

history = model.fit_generator(
        train_generator,
        steps_per_epoch=113,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=13,
        callbacks = [save_callback, save_per_epoch])

Found 3599 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/50
113/113 [==============================] - 65s 577ms/step - loss: 2.6301 - acc: 0.8256 - val_loss: 2.8610 - val_acc: 0.8225
Epoch 2/50
113/113 [==============================] - 60s 534ms/step - loss: 2.7163 - acc: 0.8315 - val_loss: 2.8610 - val_acc: 0.8225
Epoch 3/50
113/113 [==============================] - 62s 550ms/step - loss: 2.7113 - acc: 0.8318 - val_loss: 2.8610 - val_acc: 0.8225
Epoch 4/50
113/113 [==============================] - 62s 551ms/step - loss: 2.7113 - acc: 0.8318 - val_loss: 2.8610 - val_acc: 0.8225
Epoch 5/50
113/113 [==============================] - 62s 550ms/step - loss: 2.7063 - acc: 0.8321 - val_loss: 2.8610 - val_acc: 0.8225
Epoch 6/50
113/113 [==============================] - 62s 552ms/step - loss: 2.7163 - acc: 0.8315 - val_loss: 2.8610 - val_acc: 0.8225
Epoch 7/50
113/113 [==============================] - 63s 555ms/step - loss: 2.7063 - acc: 0.8321 - val_los

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()